In [1]:
import pandas as pd
import gensim
from tqdm import tqdm
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
tqdm.pandas()
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /Users/ange/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ange/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
news_df = pd.read_csv("news_data/news_data_preprocessed.csv")

In [4]:
news_df

,abstract,web_url,snippet,pub_date,document_type,news_desk,type_of_material,word_count,headline.main,headline_lemmas,snippet_lemmas
0,President Trump’s aides were delighted that ne...,https://www.nytimes.com/2019/11/30/us/politics...,President Trump’s aides were delighted that ne...,2019-12-01T00:15:08+0000,article,Washington,News,1341,A Leak-Prone White House Finally Manages to Ke...,"['a', 'white', 'house', 'finally', 'manages', ...","['president', 'trump', 'aide', 'delighted', 'n..."
1,The Buckeyes quarterback threw one of his four...,https://www.nytimes.com/2019/11/30/sports/ncaa...,The Buckeyes quarterback threw one of his four...,2019-12-01T00:33:57+0000,article,Sports,News,727,Justin Fields Has ‘Heisman Moment’ in Ohio Sta...,"['justin', 'field', 'ha', 'heisman', 'moment',...","['the', 'buckeye', 'quarterback', 'threw', 'on..."
2,A personal loss has prompted Eileen Shiffrin t...,https://www.nytimes.com/2019/11/30/sports/skii...,A personal loss has prompted Eileen Shiffrin t...,2019-12-01T00:37:15+0000,article,Sports,News,762,Mikaela Shiffrin Learns a New Way to Win: With...,"['mikaela', 'shiffrin', 'learns', 'new', 'way'...","['a', 'personal', 'loss', 'prompted', 'eileen'..."
3,The actor and environmentalist released a stat...,https://www.nytimes.com/2019/11/30/world/ameri...,The actor and environmentalist released a stat...,2019-12-01T01:03:22+0000,article,Express,News,540,Leonardo DiCaprio Responds to Brazil’s Preside...,"['leonardo', 'dicaprio', 'responds', 'brazil',...","['the', 'actor', 'environmentalist', 'released..."
4,A blocked pipe caused sewage to back up severa...,https://www.nytimes.com/2019/11/30/nyregion/Qu...,A blocked pipe caused sewage to back up severa...,2019-12-01T01:47:35+0000,article,Express,News,781,Cooking Grease Down a Drain Eyed in Sewage Flo...,"['cooking', 'grease', 'down', 'drain', 'eyed',...","['a', 'blocked', 'pipe', 'caused', 'sewage', '..."
...,...,...,...,...,...,...,...,...,...,...,...
242015,"Anna Wintour, Condé Nast’s artistic director, ...",https://www.nytimes.com/2016/02/01/business/me...,"Anna Wintour, Condé Nast’s artistic director, ...",2016-01-31T23:09:07+0000,article,Business,News,1348,"Condé Nast Adapts to New Forces, Leaving Some ...","['condé', 'nast', 'adapts', 'new', 'force', 'l...","['anna', 'wintour', 'condé', 'nast', 'artistic..."
242016,Microsoft sank a data center on the ocean floo...,https://www.nytimes.com/2016/02/01/technology/...,Microsoft sank a data center on the ocean floo...,2016-01-31T23:25:58+0000,article,Business,News,1223,Microsoft Plumbs Ocean’s Depths to Test Underw...,"['microsoft', 'plumb', 'ocean', 'depth', 'test...","['microsoft', 'sank', 'data', 'center', 'ocean..."
242017,"David Eisenhauer, a freshman charged with abdu...",https://www.nytimes.com/2016/02/01/us/two-virg...,"David Eisenhauer, a freshman charged with abdu...",2016-01-31T23:35:00+0000,article,National,News,459,Two Virginia Tech Students Arrested in Girl’s ...,"['two', 'virginia', 'tech', 'student', 'arrest...","['david', 'eisenhauer', 'freshman', 'charged',..."
242018,A time-capsule look at how the show as markete...,https://artsbeat.blogs.nytimes.com/2016/01/31/...,A time-capsule look at how the show as markete...,2016-01-31T23:53:20+0000,article,Culture,News,161,Theater Flashback: ‘Grease’ on Broadway,"['theater', 'flashback', 'grease', 'broadway']","['a', 'look', 'show', 'marketed', 'first', 'op..."


In [5]:
# Construct a dictionary of words from the corpus
dictionary = gensim.corpora.Dictionary(news_df['headline_lemmas'])
print(f"Before filtering: {len(dictionary)}")

# Filter the dictionary to meet frequency thresholds
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=10000)
print(f"After filtering: {len(dictionary)}")

corpus = [dictionary.doc2bow(doc) for doc in news_df['headline_lemmas']]
id2token = {id: token for token, id in dictionary.token2id.items()}

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [6]:
from gensim.models import LdaMulticore
lda = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
topics = lda.show_topics(num_topics=10, num_words=10, log=True, formatted=False)

# print the top words associated with each topic (top 10)
for topic_num, topic_words in topics:
  topic_rep = " ".join ([f"{w}({p:.4f})" for w,p in topic_words])
  print (f"Topic {topic_num}: {topic_rep}")

NameError: name 'corpus' is not defined